In [1]:
import json
import copy
import random
import numpy as np
import pandas as pd
import math
from datetime import datetime

In [43]:
time_slots = range(0, 65)
k = 5

In [116]:
lecturers = pd.DataFrame(np.array([['Kartelj'], ['Cukic'], ['Mitic'], ['Malkov']]),
                   columns=['name'])
preffered_timeslots = [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],[],[],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]]
lecturers['preffered_timeslots'] = preffered_timeslots
not_preffered_timeslots = [[24,25,26,27,28,29,30,31,32,33,34,35],[],[],[36,37,38,39,40,41,42,43,44,45,46,47]]
lecturers['not_preffered_timeslots'] = not_preffered_timeslots
preffered_locations = [{'1', '2'},set(),set(),{'0'}]
lecturers['preffered_locations'] = preffered_locations
lecturers

,name,preffered_timeslots,not_preffered_timeslots,preffered_locations
0,Kartelj,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","{2, 1}"
1,Cukic,[],[],{}
2,Mitic,[],[],{}
3,Malkov,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]",{0}


In [117]:
classes = pd.DataFrame(np.array([['Racunarske mreze', None, 2], ['Racunarske mreze - vezbe', 'racunari', 3], 
                                 ['Racunarska inteligencija', None, 2], ['Racunarska inteligencija - vezbe', 'racunari', 3], 
                                 ['Funkcionalno programiranje', None, 2], ['Funkcionalno programiranje - vezbe', 'racunari', 3],
                                 ['Programiranje za veb', None, 2], ['Programiranje za veb - vezbe', 'racunari', 3],
                                 ['Istrazivanje podataka 2', None, 2], ['Istrazivanje podataka 2 - vezbe', 'racunari', 3],
                                 ['Razvoj sofvtvera', None, 2], ['Razvoj softvera - vezbe', 'racunari', 3],
                                 ['Projektovanje baza podataka', None, 2], ['Projektovanje baza podataka - vezbe', 'racunari', 3],
                                ]), columns=['name', 'equipment', 'number_of_classes'])
classes

,name,equipment,number_of_classes
0,Racunarske mreze,None,2
1,Racunarske mreze - vezbe,racunari,3
2,Racunarska inteligencija,None,2
3,Racunarska inteligencija - vezbe,racunari,3
4,Funkcionalno programiranje,None,2
5,Funkcionalno programiranje - vezbe,racunari,3
6,Programiranje za veb,None,2
7,Programiranje za veb - vezbe,racunari,3
8,Istrazivanje podataka 2,None,2
9,Istrazivanje podataka 2 - vezbe,racunari,3


In [118]:
groups = pd.DataFrame(np.array([['4i', None, False, {'1', '2', '3', '4'}], ['4i1a', '4i', True, None], ['4i1b', '4i', True, None], ['4i2a', '4i', True, None], ['4i2b', '4i', True, None],
                                ['4r', None, False, {'6', '7', '8'}], ['r4gr1', '4r', True, None], ['r4gr2', '4r', True, None], ['r4gr3', '4r', True, None]
                               ]), columns=['name', 'parent', 'main', 'children'])
groups

,name,parent,main,children
0,4i,None,False,"{2, 1, 4, 3}"
1,4i1a,4i,True,None
2,4i1b,4i,True,None
3,4i2a,4i,True,None
4,4i2b,4i,True,None
5,4r,None,False,"{7, 6, 8}"
6,r4gr1,4r,True,None
7,r4gr2,4r,True,None
8,r4gr3,4r,True,None


In [2]:
locations = pd.DataFrame(np.array([['Studentski trg'], ['Sv. Nikole'], ['Jagic']]), columns=['name'])
locations

,name
0,Studentski trg
1,Sv. Nikole
2,Jagic


In [3]:
location_distances = pd.DataFrame(np.array([['0', '1', 1], ['0', '2', 1], ['1', '2', 1]]), columns=['first', 'second', 'hours'])
location_distances

,first,second,hours
0,0,1,1
1,0,2,1
2,1,2,1


In [121]:
rooms = pd.DataFrame(np.array([['704', '0', 'racunari'], ['718', '0', 'racunari'], ['706', '0', None], ['821', '0', None],
                               ['BIM', '0', 'racunari'], ['RLAB', '0', 'racunari'], ['830', '0', None], ['N201', '1', 'racunari'],
                               ['N202', '1', 'racunari'], ['N253', '1', None], ['Jag1', '2', 'racunari'],
                               ['Jag2', '2', 'racunari'], ['Jag3', '2', None], ['Jag4', '2', None]
                              ]), columns=['name', 'location', 'equipment'])
rooms

,name,location,equipment
0,704,0,racunari
1,718,0,racunari
2,706,0,None
3,821,0,None
4,BIM,0,racunari
5,RLAB,0,racunari
6,830,0,None
7,N201,1,racunari
8,N202,1,racunari
9,N253,1,None


In [122]:
groups

,name,parent,main,children
0,4i,None,False,"{2, 1, 4, 3}"
1,4i1a,4i,True,None
2,4i1b,4i,True,None
3,4i2a,4i,True,None
4,4i2b,4i,True,None
5,4r,None,False,"{7, 6, 8}"
6,r4gr1,4r,True,None
7,r4gr2,4r,True,None
8,r4gr3,4r,True,None


In [123]:
teaching_curriculum = pd.DataFrame(np.array([['0', '0', {'0'}, {'0','1','2','3','4'}], 
                                             ['0', '1', {'1', '2'}, {'0','1','2'}], 
                                             ['0', '1', {'3', '4'}, {'0','3','4'}], 
                                             ['3', '10', {'1', '2'}, {'0','1','2'}],
                                             ['3', '10', {'3', '4', '5'}, {'0','3','4','5','6','7','8'}], 
                                             ['3', '11', {'1'}, {'0','1'}], 
                                             ['3', '11', {'2'}, {'0','2'}], 
                                             ['3', '11', {'3', '5'}, {'0','3','5','6','7','8'}],
                                             ['3', '11', {'4'}, {'0','4'}], 
                                             ['2', '8', {'0'}, {'0','1','2','3','4'}], 
                                             ['2', '9', {'1', '2'}, {'0','1','2',}], 
                                             ['2', '9', {'3', '4'}, {'0','3','4'}]                                               
                                            ]), columns=['lecturer', 'class', 'groups', 'groups_to_check'])
teaching_curriculum

,lecturer,class,groups,groups_to_check
0,0,0,{0},"{1, 0, 3, 2, 4}"
1,0,1,"{2, 1}","{2, 1, 0}"
2,0,1,"{4, 3}","{0, 4, 3}"
3,3,10,"{2, 1}","{2, 1, 0}"
4,3,10,"{5, 4, 3}","{7, 6, 0, 3, 5, 4, 8}"
5,3,11,{1},"{1, 0}"
6,3,11,{2},"{2, 0}"
7,3,11,"{5, 3}","{7, 6, 0, 3, 5, 8}"
8,3,11,{4},"{0, 4}"
9,2,8,{0},"{1, 0, 3, 2, 4}"


In [124]:
teaching_curriculum_copy = teaching_curriculum.copy()
teaching_curriculum_copy['lecturer'] = lecturers.iloc[teaching_curriculum_copy['lecturer'].tolist()].name.tolist()
teaching_curriculum_copy['class'] = classes.iloc[teaching_curriculum_copy['class'].tolist()].name.tolist()
teaching_curriculum_copy['groups'] = list(map(lambda gs: list(map(lambda g: groups.iloc[int(g)]['name'], gs)), teaching_curriculum_copy['groups'].tolist()))
teaching_curriculum_copy

,lecturer,class,groups,groups_to_check
0,Kartelj,Racunarske mreze,[4i],"{1, 0, 3, 2, 4}"
1,Kartelj,Racunarske mreze - vezbe,"[4i1b, 4i1a]","{2, 1, 0}"
2,Kartelj,Racunarske mreze - vezbe,"[4i2b, 4i2a]","{0, 4, 3}"
3,Malkov,Razvoj sofvtvera,"[4i1b, 4i1a]","{2, 1, 0}"
4,Malkov,Razvoj sofvtvera,"[4r, 4i2b, 4i2a]","{7, 6, 0, 3, 5, 4, 8}"
5,Malkov,Razvoj softvera - vezbe,[4i1a],"{1, 0}"
6,Malkov,Razvoj softvera - vezbe,[4i1b],"{2, 0}"
7,Malkov,Razvoj softvera - vezbe,"[4r, 4i2a]","{7, 6, 0, 3, 5, 8}"
8,Malkov,Razvoj softvera - vezbe,[4i2b],"{0, 4}"
9,Mitic,Istrazivanje podataka 2,[4i],"{1, 0, 3, 2, 4}"


In [56]:
#new data
df = pd.read_csv('./files/2022_2023.csv')

set_of_lecturers = set()
for i in range(df.shape[0]):
    if df.iloc[i]['Предавања'] == df.iloc[i]['Предавања']:
        set_of_lecturers.add(df.iloc[i]['Предавања'])
    if df.iloc[i]['Вежбе'] == df.iloc[i]['Вежбе']:
        set_of_lecturers.add(df.iloc[i]['Вежбе'])
lecturers = pd.DataFrame(set_of_lecturers, columns=["name"])
lecturers['preffered_timeslots'] = np.empty((len(lecturers), 0)).tolist()
lecturers['not_preffered_timeslots'] = np.empty((len(lecturers), 0)).tolist()
lecturers['preffered_locations'] = np.empty((len(lecturers), 0)).tolist()

set_of_classes = set()
for i in range(df.shape[0]):
    if (df.iloc[i]['Шифра'] == df.iloc[i]['Шифра']) and (df.iloc[i]['Фонд предавања'] == df.iloc[i]['Фонд предавања']):
        set_of_classes.add((df.iloc[i]['Шифра'], None, int(df.iloc[i]['Фонд предавања'])))
    if df.iloc[i]['Групе В'] == df.iloc[i]['Групе В'] and int(df.iloc[i]['Групе В']) > 0 and (df.iloc[i]['Фонд вежбе'] == df.iloc[i]['Фонд вежбе']):
        set_of_classes.add((str(df.iloc[i]['Шифра']) + ' - вежбе', None, int(df.iloc[i]['Фонд вежбе'])))
classes = pd.DataFrame(set_of_classes, columns=['name', 'equipment', 'number_of_classes'])

set_of_groups = set()
for i in range(df.shape[0]):
    if (df.iloc[i]['Ток'] == df.iloc[i]['Ток']):
        set_of_groups.add((df.iloc[i]['Ток']))
    if df.iat[i,13] == df.iat[i,13]:
        set_of_groups.add(df.iat[i,13])
groups = pd.DataFrame(set_of_groups, columns=['name'])
groups['parent'] = None

teaching_curriculum = pd.DataFrame([], columns=['lecturer', 'class', 'groups', 'groups_to_check'])
for i in range(df.shape[0]):
    if (df.iloc[i]['Шифра'] == df.iloc[i]['Шифра']) and (df.iloc[i]['Фонд предавања'] == df.iloc[i]['Фонд предавања']) and (df.iloc[i]['Ток'] == df.iloc[i]['Ток']) and (df.iloc[i]['Предавања'] == df.iloc[i]['Предавања']):
        teaching_curriculum.loc[len(teaching_curriculum.index)] = [str(lecturers[lecturers['name']==df.iloc[i]['Предавања']].index[0]), str(classes[classes['name'] == df.iloc[i]['Шифра']].index[0]), {str(groups[groups['name']==df.iloc[i]['Ток']].index[0])}, {str(groups[groups['name']==df.iloc[i]['Ток']].index[0])}]

In [56]:
groups

,name,parent,main,children,focus
0,1o1,NaN,False,"{'2', '1'}","{1o1, 1o1a, 1o1b}"
1,1o1a,1o1,True,NaN,"{1o1a, 1o1}"
2,1o1b,1o1,True,NaN,"{1o1, 1o1b}"
3,1o2,NaN,False,"{'5', '4'}","{1o2, 1o2a, 1o2b}"
4,1o2a,1o2,True,NaN,"{1o2, 1o2a}"
...,...,...,...,...,...
85,4v,NaN,True,NaN,{4v}
86,4m,NaN,True,NaN,{4m}
87,5m,NaN,True,NaN,{5m}
88,5l,NaN,True,NaN,{5l}


In [55]:
teaching_curriculum = pd.read_csv('teaching_curriculum.csv', index_col=0)
groups = pd.read_csv('groups.csv', index_col=0)
lecturers = pd.read_csv('lecturers.csv', index_col=0)
classes = pd.read_csv('classes.csv', index_col=0)
rooms = pd.read_csv('rooms.csv', index_col=0)
teaching_curriculum['groups'] = teaching_curriculum['groups'].apply(eval)
teaching_curriculum['groups_to_check'] = teaching_curriculum['groups_to_check'].apply(eval)
groups['focus'] = groups['focus'].apply(eval)
lecturers['preffered_timeslots'] = lecturers['preffered_timeslots'].apply(eval)
lecturers['not_preffered_timeslots'] = lecturers['not_preffered_timeslots'].apply(eval)
lecturers['preffered_locations'] = lecturers['preffered_locations'].apply(eval)
teaching_curriculum['lecturer'] = teaching_curriculum['lecturer'].astype('str')
teaching_curriculum['class'] = teaching_curriculum['class'].astype('str')

In [53]:
classes

,name,equipment,number_of_classes
0,Р272,NaN,2
1,РМ09,NaN,3
2,Р210,NaN,3
3,Р274,NaN,3
4,Р309 - вежбе,NaN,3
...,...,...,...
143,Р309,NaN,2
144,МФУБИ,NaN,2
145,РМ18,NaN,3
146,15-РМ06 - вежбе,NaN,2


In [4]:
print(type(teaching_curriculum.iat[0,1]))

<class 'str'>


In [5]:
teaching_curriculum

,lecturer,class,groups,groups_to_check
0,15,55,{0},"{1, 2, 0}"
1,37,18,{1},"{1, 0}"
2,37,18,{2},"{2, 0}"
3,21,18,{4},"{4, 3}"
4,21,18,{5},"{5, 3}"
...,...,...,...,...
249,9,7,{89},{89}
250,45,79,{89},{89}
251,10,47,{89},{89}
252,45,104,{89},{89}


In [31]:
time_slots = range(0, 65)

In [5]:
def init_timetable():
    init = []
    for i in range(teaching_curriculum.shape[0]):
        init.append([rooms.iloc[i % len(rooms)].name, time_slots[i // len(rooms)]])
    timetable = pd.DataFrame(np.array(init), columns = ['room', 'time_slot'])
    return timetable

In [6]:
def are_classes_intersected(timetable,teaching_curriculum, i, j):
    first = timetable.iat[i, 1] + classes.iat[int(teaching_curriculum.iat[i, 1]), 2] <= timetable.iat[j, 1]
    second = timetable.iat[j, 1] + classes.iat[int(teaching_curriculum.iat[j, 1]), 2] <= timetable.iat[i, 1]
    return not (first or second)

def are_locations_intersected(timetable,teaching_curriculum, i, j):
    i_location = rooms.iat[int(timetable.iat[i,0]),1]
    j_location = rooms.iat[int(timetable.iat[j,0]),1]
    if (i_location == j_location) or ((timetable.iat[i, 1] // 13) != (timetable.iat[j, 1] // 13)):
        return False
    first_location = min(int(i_location), int(j_location))
    second_location = max(int(i_location), int(j_location))
    distance_in_hours = int(location_distances[(location_distances['first'] == str(first_location)) & (location_distances['second'] == str(second_location))].iat[0,2])
    first = abs(timetable.iat[i,1] + classes.iat[int(teaching_curriculum.iat[i, 1]), 2] - timetable.iat[j, 1]) >= distance_in_hours 
    second = abs(timetable.iat[j,1] + classes.iat[int(teaching_curriculum.iat[j,1]), 2] - timetable.iat[i, 1]) >= distance_in_hours
    return not (first and second)

def are_groups_intersected(teaching_curriculum, i, j):
    set1 = teaching_curriculum.iat[i,2]
    set2 = teaching_curriculum.iat[j,2]
    whole_set1 = teaching_curriculum.iat[i,3]
    whole_set2 = teaching_curriculum.iat[j,3]
    for el1 in set1:
        for el2 in set2:
            if el1 == el2 or el1 in whole_set2 or el2 in whole_set1:
                return True
    return False

In [9]:
calculate_soft_constraints(result_timetable, teaching_curriculum, [80,81])

NameError: name 'calculate_soft_constraints' is not defined

In [7]:
def calculate_soft_constraints(timetable, teaching_curriculum, indexes):
    list_of_timeslots = [set() for x in range(65)]
    for i in indexes:
        timeslot = timetable.iat[i,1]
        num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
        for j in range(timeslot, timeslot + num_of_classes):
            list_of_timeslots[j].add(rooms.iat[int(timetable.iat[i,0]),1])
    count = 0
#     print('list', list_of_timeslots)
    last_set = set()
    last_index = 0
    for i in range(len(list_of_timeslots)):
        if i % 13 == 0:
            last_set = set()
        if last_set and last_index < i - 1 and list_of_timeslots[i]:
            count += 1
        if last_set and list_of_timeslots[i] and last_set.isdisjoint(list_of_timeslots[i]):
            count += 1
        if (list_of_timeslots[i]):
            last_set = list_of_timeslots[i]
            last_index = i
    return count

def calculate_lecturer_preferences(timetable, teaching_curriculum, indexes):
    list_of_timeslots = [False for x in range(65)]
    list_of_locations = set()
    lecturer = lecturers.iloc[int(teaching_curriculum.iat[indexes[0],0])]
    for i in indexes:
        timeslot = timetable.iat[i,1]
        num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
        list_of_locations.add(rooms.iat[int(timetable.iat[i,0]),1])
        for j in range(timeslot, timeslot + num_of_classes):
            list_of_timeslots[j] = True
    count = 0
    if lecturer['preffered_locations'] and list_of_locations.difference(lecturer['preffered_locations']):
        count += 1
    for i in range(len(lecturer['not_preffered_timeslots'])):
        if list_of_timeslots[lecturer['not_preffered_timeslots'][i]]:
            count += 1
    if lecturer['preffered_timeslots']:
        for i in range(len(list_of_timeslots)):
            if list_of_timeslots[i] and i not in lecturer['preffered_timeslots']:
                count += 1
    return count

In [64]:
def lecturer_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if teaching_curriculum.iat[i,0] == teaching_curriculum.iat[j,0] and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

def lecturer_constraint_delta(timetable, teaching_curriculum, i):
    lecturer = teaching_curriculum.iat[i,0]
    teaching_curriculum_with_lecturer = teaching_curriculum[teaching_curriculum['lecturer'] == lecturer]
    count = 0
    for j in teaching_curriculum_with_lecturer.index:
        if i != j:
            if (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

In [65]:
def group_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if are_groups_intersected(teaching_curriculum, i, j) and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

def group_constraint_delta(timetable, teaching_curriculum, i):
    count = 0
    for j in range(teaching_curriculum.shape[0]):
        if i != j:
            if are_groups_intersected(teaching_curriculum, i, j) and (are_classes_intersected(timetable, teaching_curriculum, i, j) or are_locations_intersected(timetable, teaching_curriculum, i, j)):
                count += 1
    return count

In [66]:
def room_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        for j in range(i+1, teaching_curriculum.shape[0]):
            if timetable.iat[i,0] == timetable.iat[j,0] and are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

def room_constraint_delta(timetable, teaching_curriculum, i):
    room = timetable.iat[i,0]
    timetable_with_room = timetable[timetable['room'] == room]
    count = 0
    for j in timetable_with_room.index:
        if i != j:
            if are_classes_intersected(timetable, teaching_curriculum, i, j):
                count += 1
    return count

In [52]:
classes

,name,equipment,number_of_classes
0,Р272,NaN,2
1,РМ09,NaN,3
2,Р210,NaN,3
3,Р274,NaN,3
4,Р309 - вежбе,NaN,3
...,...,...,...
143,Р309,NaN,2
144,МФУБИ,NaN,2
145,РМ18,NaN,3
146,15-РМ06 - вежбе,NaN,2


In [62]:
def equipment_constraint(timetable, teaching_curriculum):
    count = 0
    for i in range(teaching_curriculum.shape[0]):
        if classes.iat[int(teaching_curriculum.iat[i,1]),1] == classes.iat[int(teaching_curriculum.iat[i,1]),1] and classes.iat[int(teaching_curriculum.iat[i,1]),1] is not None and classes.iat[int(teaching_curriculum.iat[i,1]),1] != rooms.iat[int(timetable.iat[i,0]),2]:
            count += 1
    return count

def equipment_constraint_delta(timetable, teaching_curriculum, i):
    count = 0
    if classes.iat[int(teaching_curriculum.iat[i,1]),1] == classes.iat[int(teaching_curriculum.iat[i,1]),1] and classes.iat[int(teaching_curriculum.iat[i,1]),1] is not None and classes.iat[int(teaching_curriculum.iat[i,1]),1] != rooms.iat[int(timetable.iat[i,0]),2]:
        count += 1
    return count

In [63]:
calculate_fittnes(result_timetable, teaching_curriculum)

lecturer count 0
group count 0
room count 0


(0, 0)

In [12]:
def lecturer_soft_constraint(timetable, teaching_curriculum):
    count = 0
    for i in lecturers.index:
        indexes = teaching_curriculum[teaching_curriculum['lecturer'] == str(i)].index
        if indexes.empty:
            continue
        count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
        count += calculate_lecturer_preferences(timetable, teaching_curriculum, indexes)
    return count

def lecturer_soft_constraint_delta(timetable, teaching_curriculum, i):
    lecturer_index = teaching_curriculum.iat[i,0]
    indexes = teaching_curriculum[teaching_curriculum['lecturer'] == lecturer_index].index
    return calculate_soft_constraints(timetable, teaching_curriculum, indexes) + calculate_lecturer_preferences(timetable, teaching_curriculum, indexes)

In [16]:
indexes = teaching_curriculum[teaching_curriculum['lecturer'] == str(0)].index
indexes

Int64Index([62, 170, 192, 224], dtype='int64')

In [17]:
teaching_curriculum[teaching_curriculum['lecturer'] == str(2)]

,lecturer,class,groups,groups_to_check
54,2,81,"{50, 49}","{48, 50, 49}"
55,2,81,{51},"{51, 48}"
131,2,112,{63},"{62, 63}"
137,2,77,{70},"{68, 70}"
207,2,40,"{82, 79}","{81, 82, 84, 80, 79, 83}"
214,2,91,"{82, 79}","{81, 82, 84, 80, 79, 83}"
227,2,117,"{82, 79}","{81, 82, 84, 80, 79, 83}"


In [18]:
teaching_curriculum['lecturer']

0      15
1      37
2      37
3      21
4      21
       ..
249     9
250    45
251    10
252    45
253     9
Name: lecturer, Length: 254, dtype: object

In [13]:
def groups_soft_constraint(timetable, teaching_curriculum):
    return 0
    count = 0
    for i in groups.index:
        if groups.iat[i,2] == True:
            indexes = teaching_curriculum[(teaching_curriculum['groups_to_check'].astype(str).str.contains(str(i)))].index
            count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
    return count

def groups_soft_constraint_delta(timetable, teaching_curriculum, i):
    return 0
    group_set = teaching_curriculum.iat[i,2]
    count = 0
    for el in group_set:
        if groups.iat[int(el),2] == True:
            indexes = teaching_curriculum[(teaching_curriculum['groups_to_check'].astype(str).str.contains(str(el)))].index                               
            count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
        else:
            children_groups = groups.iat[int(el),3]
            for j in children_groups:
                indexes = teaching_curriculum[(teaching_curriculum['groups_to_check'].astype(str).str.contains(str(j)))].index 
                count += calculate_soft_constraints(timetable, teaching_curriculum, indexes)
    return count

In [ ]:
calculate_fittnes(result_timetable, teaching_curriculum)

In [20]:
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))

NameError: name 'calculate_fittnes' is not defined

In [21]:
pd.set_option('display.max_rows', 300)
result_timetable

,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [22]:
groups_soft_constraint_delta(timetable_v2, teaching_curriculum, 1)

NameError: name 'timetable_v2' is not defined

In [23]:
initTimetable = init_timetable()

In [24]:
calculate_fitness_delta(initTimetable, teaching_curriculum, 0)

NameError: name 'calculate_fitness_delta' is not defined

In [25]:
calculate_fittnes(initTimetable, teaching_curriculum)

NameError: name 'calculate_fittnes' is not defined

In [26]:
groups

,name,parent,main,children,focus
0,1o1,NaN,False,"{'1', '2'}","{1o1a, 1o1, 1o1b}"
1,1o1a,1o1,True,NaN,"{1o1a, 1o1}"
2,1o1b,1o1,True,NaN,"{1o1, 1o1b}"
3,1o2,NaN,False,"{'5', '4'}","{1o2b, 1o2, 1o2a}"
4,1o2a,1o2,True,NaN,"{1o2, 1o2a}"
5,1o2b,1o2,True,NaN,"{1o2b, 1o2}"
6,1o3,NaN,False,"{'8', '7'}","{1o3b, 1o3, 1o3a}"
7,1o3a,1o3,True,NaN,"{1o3, 1o3a}"
8,1o3b,1o3,True,NaN,"{1o3b, 1o3}"
9,1o4,NaN,False,"{'10', '11'}","{1o4b, 1o4, 1o4a}"


In [27]:
teaching_curriculum

,lecturer,class,groups,groups_to_check
0,15,55,{0},"{1, 2, 0}"
1,37,18,{1},"{1, 0}"
2,37,18,{2},"{2, 0}"
3,21,18,{4},"{4, 3}"
4,21,18,{5},"{5, 3}"
5,37,18,{7},"{7, 6}"
6,42,18,{8},"{8, 6}"
7,7,18,{10},"{9, 10}"
8,7,18,{11},"{11, 9}"
9,27,123,{0},"{1, 2, 0}"


In [14]:
def calculate_fittnes(timetable, teaching_curriculum):
    first = lecturer_constraint(timetable, teaching_curriculum) + group_constraint(timetable, teaching_curriculum) + room_constraint(timetable, teaching_curriculum) + equipment_constraint(timetable, teaching_curriculum)
    second = lecturer_soft_constraint(timetable, teaching_curriculum)+ groups_soft_constraint(timetable, teaching_curriculum)
    return (first, second)

def calculate_fitness_delta(timetable, teaching_curriculum, i):
    first = lecturer_constraint_delta(timetable, teaching_curriculum, i) + group_constraint_delta(timetable, teaching_curriculum, i) + room_constraint_delta(timetable, teaching_curriculum, i) + equipment_constraint_delta(timetable, teaching_curriculum, i)
    second = lecturer_soft_constraint_delta(timetable, teaching_curriculum, i) + groups_soft_constraint_delta(timetable, teaching_curriculum, i)
    return (first, second)

In [29]:
pd.DataFrame(False, index=np.arange(lecturers.shape[0]), columns=time_slots)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
def create_constraint_matrixes(timetable, teaching_curriculum):
    lecturer_constraint_matrix = pd.DataFrame(False, index=np.arange(lecturers.shape[0]), columns=time_slots)
    groups_constraint_matrix = pd.DataFrame(False, index=np.arange(groups.shape[0]), columns=time_slots)
    rooms_constraint_matrix = pd.DataFrame(False, index=np.arange(rooms.shape[0]), columns=time_slots)
    for i in range(timetable.shape[0]):
        timeslot = timetable.iat[i,1]
        num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
        for j in range(timeslot, timeslot + num_of_classes):
            lecturer_constraint_matrix.iat[int(teaching_curriculum.iat[i,0]),j] = True
            for group in teaching_curriculum.iat[i,2]:
                groups_constraint_matrix.iat[int(group),j] = True
            rooms_constraint_matrix.iat[int(timetable.iat[i,0]),j] = True
    return (lecturer_constraint_matrix, groups_constraint_matrix, rooms_constraint_matrix)

In [31]:
groups

,name,parent,main,children,focus
0,1o1,NaN,False,"{'1', '2'}","{1o1a, 1o1, 1o1b}"
1,1o1a,1o1,True,NaN,"{1o1a, 1o1}"
2,1o1b,1o1,True,NaN,"{1o1, 1o1b}"
3,1o2,NaN,False,"{'5', '4'}","{1o2b, 1o2, 1o2a}"
4,1o2a,1o2,True,NaN,"{1o2, 1o2a}"
5,1o2b,1o2,True,NaN,"{1o2b, 1o2}"
6,1o3,NaN,False,"{'8', '7'}","{1o3b, 1o3, 1o3a}"
7,1o3a,1o3,True,NaN,"{1o3, 1o3a}"
8,1o3b,1o3,True,NaN,"{1o3b, 1o3}"
9,1o4,NaN,False,"{'10', '11'}","{1o4b, 1o4, 1o4a}"


In [32]:
groups.iat[1,1]

'1o1'

In [33]:
teaching_curriculum.iat[1,2]

{'1'}

In [16]:
def check_for_constraints(timetable, teaching_curriculum, constraint_matrixes, i, timeslot):
    (lecturer_constraint_matrix, groups_constraint_matrix, rooms_constraint_matrix) = constraint_matrixes
    old_timeslot = timetable.iat[i,1]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    old_place = range(old_timeslot, old_timeslot + num_of_classes)
    set_of_groups = set()
    for group in teaching_curriculum.iat[i,2]:
        set_of_groups.add(group)
        if groups.iat[int(group),1]:
            set_of_groups.add(groups[groups['name'] == groups.iat[int(group),1]].index[0])
    for j in range(timeslot, timeslot + num_of_classes):
        if j in old_place:
            continue
        if lecturer_constraint_matrix.iat[int(teaching_curriculum.iat[i,0]),j]:
            return True
        for group in set_of_groups:
            if groups_constraint_matrix.iat[int(group),j]:
                return True
        if rooms_constraint_matrix.iat[int(timetable.iat[i,0]),j]:
            return True
    return False

In [17]:
def check_for_constraints_room(timetable, teaching_curriculum, constraint_matrixes, i, room):
    (lecturer_constraint_matrix, groups_constraint_matrix, rooms_constraint_matrix) = constraint_matrixes
    timeslot = timetable.iat[i,1]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    for j in range(timeslot, timeslot + num_of_classes):
        if rooms_constraint_matrix.iat[int(room),j]:
            return True
    return False

In [36]:
constraint_matrixes = create_constraint_matrixes(result_timetable, teaching_curriculum)
check_for_constraints_room(result_timetable, teaching_curriculum, constraint_matrixes, int('2'), '0')

True

In [40]:
calculate_fittnes(result_timetable, teaching_curriculum)

(2560, 53)

In [41]:
result_timetable

,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [18]:
def local_search(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    while(True):
        for i in range(timetable.shape[0]):
            timeslot_range = range(12 * (timetable.iat[i,1] // 12), 12 * (timetable.iat[i,1] // 12 + 1) - classes.iat[int(teaching_curriculum.iat[i,1]),2] + 1)
            for timeslot in timeslot_range:
                current_timetable = timetable.copy()
                current_timetable.iat[i,1] = timeslot
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
            for j in range(rooms.shape[0]):
                current_timetable = timetable.copy()
                current_timetable.iat[i,0] = str(j)
                current_fittness = calculate_fittnes(current_timetable, teaching_curriculum)
                if current_fittness < best_fitness:
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
        if best_fitness < calculate_fittnes(timetable, teaching_curriculum):
            print(best_fitness)
            timetable = best_timetable.copy()
        else:
            break
    return timetable

In [19]:
def local_search_v2(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    fitness = best_fitness
    helper_matrixes = create_helper_matrixes(timetable, teaching_curriculum)
    while(True):
        for i in range(timetable.shape[0]):
#             timeslot_range = range(12 * (timetable.iat[i,1] // 12), 12 * (timetable.iat[i,1] // 12 + 1) - classes.iat[int(teaching_curriculum.iat[i,1]),2] + 1)
            for timeslot in time_slots:
                if timeslot % 13 + classes.iat[int(teaching_curriculum.iat[i,1]),2] > 12:
                    continue
                current_timetable = timetable.copy()
                current_timetable.iat[i,1] = timeslot
                first_fitness_delta = 0
                first_fitness_delta += check_lecturer_constraint_delta(timetable, teaching_curriculum, helper_matrixes[0], i, timeslot)
                first_fitness_delta += check_groups_constraint_delta(timetable, teaching_curriculum, helper_matrixes[1], i, timeslot)
                first_fitness_delta += check_room_constraint_delta(timetable, teaching_curriculum, helper_matrixes[2], i, timeslot)
                first_fitness_delta += equipment_constraint_delta(timetable, teaching_curriculum, i)
                first_currfitness_delta = 0
                first_currfitness_delta += check_lecturer_constraint_delta_curr(timetable, teaching_curriculum, helper_matrixes[0], i, timeslot)
                first_currfitness_delta += check_groups_constraint_delta_curr(timetable, teaching_curriculum, helper_matrixes[1], i, timeslot)
                first_currfitness_delta += check_room_constraint_delta_curr(timetable, teaching_curriculum, helper_matrixes[2], i, timeslot)
                first_currfitness_delta += equipment_constraint_delta(current_timetable, teaching_curriculum, i)
                if fitness[0] - first_fitness_delta + first_currfitness_delta > best_fitness[0]:
                    continue
                second_fitness_delta = 0
                second_fitness_delta += lecturer_soft_constraint_delta(timetable, teaching_curriculum, i)
                second_fitness_delta += groups_soft_constraint_delta(timetable, teaching_curriculum, i)
                second_currfitness_delta = 0
                second_currfitness_delta += lecturer_soft_constraint_delta(current_timetable, teaching_curriculum, i)
                second_currfitness_delta += groups_soft_constraint_delta(current_timetable, teaching_curriculum, i)
                fitness_delta = (first_fitness_delta, second_fitness_delta)
                current_fitness_delta = (first_currfitness_delta, second_currfitness_delta)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] == best_fitness[0]:
                    a = 'a'
                if current_fittness[0] > best_fitness[0]:
                    b = 'b'
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
                if best_fitness[1] < 0:
                    print('less than zero')
                    print(fitness[1], fitness_delta[1], current_fitness_delta[1])
                    print(i, timeslot, timetable.iat[i,1])
                    return current_timetable
                    break
            for j in range(rooms.shape[0]):
                current_timetable = timetable.copy()
                current_timetable.iat[i,0] = str(j)
                first_fitness_delta = 0
                first_fitness_delta += check_lecturer_constraint_delta_room(timetable, teaching_curriculum, helper_matrixes[0], i, str(j))
                first_fitness_delta += check_groups_constraint_delta_room(timetable, teaching_curriculum, helper_matrixes[1], i, str(j))
                first_fitness_delta += check_room_constraint_delta_room(timetable, teaching_curriculum, helper_matrixes[2], i, str(j))
                first_fitness_delta += equipment_constraint_delta(timetable, teaching_curriculum, i)
                first_currfitness_delta = 0
                first_currfitness_delta += check_lecturer_constraint_delta_curr_room(timetable, teaching_curriculum, helper_matrixes[0], i, str(j))
                first_currfitness_delta += check_groups_constraint_delta_curr_room(timetable, teaching_curriculum, helper_matrixes[1], i, str(j))
                first_currfitness_delta += check_room_constraint_delta_curr_room(timetable, teaching_curriculum, helper_matrixes[2], i, str(j))
                first_currfitness_delta += equipment_constraint_delta(current_timetable, teaching_curriculum, i)
                if fitness[0] - first_fitness_delta + first_currfitness_delta > best_fitness[0]:
                    continue
                second_fitness_delta = 0
                second_fitness_delta += lecturer_soft_constraint_delta(timetable, teaching_curriculum, i)
                second_fitness_delta += groups_soft_constraint_delta(timetable, teaching_curriculum, i)
                second_currfitness_delta = 0
                second_currfitness_delta += lecturer_soft_constraint_delta(current_timetable, teaching_curriculum, i)
                second_currfitness_delta += groups_soft_constraint_delta(current_timetable, teaching_curriculum, i)
                fitness_delta = (first_fitness_delta, second_fitness_delta)
                current_fitness_delta = (first_currfitness_delta, second_currfitness_delta)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] == best_fitness[0]:
                    a = 'a'
                if current_fittness[0] > best_fitness[0]:
                    b = 'b'
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
#                 if best_fitness[1] < 0:
#                     print('less than zero')
#                     print(fitness[1], fitness_delta[1], current_fitness_delta[1])
#                     break
        if best_fitness[0] < fitness[0] or (best_fitness[0] == fitness[0] and best_fitness[1] < fitness[1]):
            timetable = best_timetable.copy()
            fitness = best_fitness
            helper_matrixes = create_helper_matrixes(timetable, teaching_curriculum)
            full_fitness = calculate_fittnes(best_timetable, teaching_curriculum)
#             if (full_fitness[1] != best_fitness[1]):
#                 print('delta')
#                 print(best_fitness)
#                 print('full')
#                 print(full_fitness)
#                 break
            print(fitness)
        else:
            break
    return timetable

In [210]:
start = datetime.now()
global_count = 0
timetable_test = init_timetable()
timetable_test = local_search_v2(timetable_v2, teaching_curriculum)
end = datetime.now()
 
# find difference loop start and end time and display
tdsoftext = (end - start).total_seconds()

(2520, 53)
(2481, 53)
(2443, 53)
(2407, 51)
(2372, 51)
(2338, 51)
(2306, 49)
(2275, 48)
(2244, 47)
(2214, 47)
(2184, 47)
(2154, 47)
(2126, 47)
(2098, 47)
(2072, 47)
(2046, 48)
(2020, 49)
(1995, 49)
(1971, 49)
(1947, 49)
(1924, 48)
(1901, 48)
(1878, 48)
(1856, 48)
(1834, 48)
(1812, 48)
(1790, 48)
(1769, 48)
(1748, 48)
(1727, 48)
(1706, 48)
(1686, 48)
(1666, 48)
(1647, 46)
(1628, 45)
(1609, 44)
(1590, 43)
(1571, 43)
(1552, 43)
(1534, 42)
(1516, 41)
(1498, 41)
(1480, 41)
(1462, 41)
(1444, 41)
(1427, 39)
(1410, 38)
(1393, 38)
(1376, 38)
(1359, 39)
(1343, 37)
(1327, 36)
(1311, 36)
(1295, 36)
(1279, 37)
(1264, 36)
(1249, 35)
(1234, 35)
(1219, 35)
(1204, 35)
(1190, 35)
(1176, 35)
(1162, 35)
(1148, 35)
(1134, 35)
(1120, 35)
(1106, 35)
(1092, 36)
(1079, 34)
(1066, 34)
(1053, 34)
(1040, 34)
(1027, 34)
(1015, 32)
(1003, 32)
(991, 32)
(979, 32)
(967, 32)
(955, 32)
(944, 31)
(933, 31)
(922, 31)
(911, 31)
(900, 31)
(890, 29)
(880, 29)
(870, 29)
(860, 29)
(850, 29)
(840, 29)
(830, 29)
(820, 29)
(810,

In [205]:
lecturer_soft_constraint(timetable_test, teaching_curriculum)

    lecturer class    groups                       groups_to_check
0         15    55       {0}                             {2, 1, 0}
71        15    90  {48, 45}  {50, 51, 47, 49, 52, 46, 53, 48, 45}
165       15   143      {68}                          {68, 70, 69}
Int64Index([0, 71, 165], dtype='int64')


53

In [206]:
lecturer_soft_constraint_delta(timetable_test, teaching_curriculum, 0)

indexes Int64Index([0, 71, 165], dtype='int64')


3

In [177]:
timetable_test.iat[0,1] = 0

In [173]:
timetable_test

,room,time_slot
0,0,3
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [182]:
teaching_curriculum

,lecturer,class,groups,groups_to_check
0,15,55,{0},"{0, 1, 2}"
1,37,18,{1},"{0, 1}"
2,37,18,{2},"{0, 2}"
3,21,18,{4},"{3, 4}"
4,21,18,{5},"{3, 5}"
5,37,18,{7},"{6, 7}"
6,42,18,{8},"{8, 6}"
7,7,18,{10},"{9, 10}"
8,7,18,{11},"{9, 11}"
9,27,123,{0},"{0, 1, 2}"


In [20]:
def local_search_v3(timetable, teaching_curriculum):
    best_timetable = timetable.copy()
    best_fitness = calculate_fittnes(timetable, teaching_curriculum)
    fitness = best_fitness
    constraint_matrixes = create_constraint_matrixes(timetable, teaching_curriculum)
    while(True):
        for i in range(timetable.shape[0]):
            timeslot_range = range(12 * (timetable.iat[i,1] // 12), 12 * (timetable.iat[i,1] // 12 + 1) - classes.iat[int(teaching_curriculum.iat[i,1]),2] + 1)
            for timeslot in timeslot_range:
                if fitness[0] == 0 and check_for_constraints(timetable, teaching_curriculum, constraint_matrixes, int(i), timeslot):
                    continue
                current_timetable = timetable.copy()
                current_timetable.iat[i,1] = timeslot
                fitness_delta = calculate_fitness_delta(timetable, teaching_curriculum, i)
                current_fitness_delta = calculate_fitness_delta(current_timetable, teaching_curriculum, i)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
            for j in range(rooms.shape[0]):
                if fitness[0] == 0 and check_for_constraints_room(timetable, teaching_curriculum, constraint_matrixes, int(i), str(j)):
                    continue
                current_timetable = timetable.copy()
                current_timetable.iat[i,0] = str(j)
                fitness_delta = calculate_fitness_delta(timetable, teaching_curriculum, i)
                current_fitness_delta = calculate_fitness_delta(current_timetable, teaching_curriculum, i)
                current_fittness = (fitness[0] - fitness_delta[0] + current_fitness_delta[0], fitness[1] - fitness_delta[1] + current_fitness_delta[1])
                if current_fittness[0] < best_fitness[0] or (current_fittness[0] == best_fitness[0] and current_fittness[1] < best_fitness[1]):
                    best_timetable = current_timetable.copy()
                    best_fitness = current_fittness
        if best_fitness[0] < fitness[0] or (best_fitness[0] == fitness[0] and best_fitness[1] < fitness[1]):
            print('delta')
            print(best_fitness)
            print('full')
            print(calculate_fittnes(best_timetable, teaching_curriculum))
            timetable = best_timetable.copy()
            fitness = best_fitness
            constraint_matrixes = create_constraint_matrixes(timetable, teaching_curriculum)
        else:
            break
    return timetable

In [33]:
def shaking(timetable, teaching_curriculum, k):
    time_table_shaking = timetable.copy()
    for j in range(k):
        random_timetable_index = random.choice(range(timetable.shape[0]))
        random_room_index = random.choice(range(rooms.shape[0]))
        random_timeslot = random.choice(time_slots)
        while(random_timeslot % 13 + classes.iat[int(teaching_curriculum.iat[random_timetable_index,1]),2] > 12):
            random_timeslot = random.choice(time_slots)
        time_table_shaking.iloc[random_timetable_index] = np.array([random_room_index, random_timeslot])
    return time_table_shaking

In [129]:
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))

init (2560, 5)


In [249]:
k = 40
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
for i in range(10):
    print('iteration', i)
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search_v2(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable = timetable

init (62, 2)
iteration 0
(10, 18)
(8, 18)
(6, 20)
(5, 19)
(4, 18)
(3, 18)
(2, 18)
(1, 18)
(0, 18)
(0, 12)
(0, 11)
(0, 10)
(0, 9)
iteration 1
(5, 16)
(3, 18)
(2, 16)
(1, 16)
(0, 16)
(0, 10)
(0, 6)
iteration 2
(10, 23)
(7, 21)
(5, 20)
(4, 18)
(3, 17)
(2, 17)
(1, 17)
(0, 17)
(0, 14)
(0, 11)
(0, 7)
iteration 3
(6, 26)
(5, 20)
(4, 17)
(3, 14)
(2, 13)
(1, 13)
(0, 13)
(0, 7)
(0, 4)
iteration 4
(8, 19)
(6, 16)
(4, 14)
(2, 16)
(1, 16)
(0, 16)
(0, 11)
(0, 8)
(0, 6)
(0, 5)
iteration 5
(5, 35)
(4, 34)
(3, 33)
(2, 33)
(1, 33)
(0, 33)
(0, 25)
(0, 18)
(0, 15)
(0, 12)
(0, 10)
(0, 8)
(0, 7)
(0, 6)
(0, 5)
(0, 4)
(0, 2)
iteration 6
(6, 28)
(5, 25)
(4, 22)
(3, 21)
(2, 20)
(1, 20)
(0, 20)
(0, 11)
(0, 9)
(0, 8)
iteration 7
(3, 25)
(2, 24)
(1, 24)
(0, 24)
(0, 17)
(0, 11)
(0, 9)
(0, 8)
(0, 7)
(0, 6)
(0, 5)
(0, 4)
(0, 3)
iteration 8
(6, 21)
(4, 19)
(3, 19)
(2, 19)
(1, 19)
(0, 19)
(0, 17)
(0, 16)
(0, 15)
(0, 13)
(0, 11)
(0, 9)
(0, 8)
(0, 7)
iteration 9
(6, 16)
(4, 14)
(2, 15)
(1, 14)
(0, 14)
(0, 12)
(0, 11)
(0,

In [232]:
timetable.iloc[1].room

['0']

In [213]:
init_time = init_timetable()

In [214]:
shaking(init_time, teaching_curriculum)

time type <class 'int'>
room type <class 'int'>


,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,2,12
7,7,0
8,8,0
9,9,0


In [228]:
timetable = shaking(timetable, teaching_curriculum)

time type <class 'int'>
room type <class 'int'>


In [211]:
timetable = local_search_v2(timetable, teaching_curriculum)

(40, 2)
(29, 2)
(21, 2)
(15, 2)
(11, 2)
(8, 2)
(5, 2)
(3, 2)
(2, 2)
(1, 2)
(0, 2)
(0, 1)


In [198]:
timetable

,room,time_slot
0,0,14
1,12,6
2,2,26
3,8,48
4,0,40
5,0,16
6,1,4
7,9,12
8,10,50
9,0,1


In [27]:
teaching_curriculum_copy

,lecturer,class,groups
0,Kartelj,Racunarske mreze,[4i]
1,Kartelj,Racunarske mreze - vezbe,"[4i1b, 4i1a]"
2,Kartelj,Racunarske mreze - vezbe,"[4i2a, 4i2b]"
3,Malkov,Razvoj sofvtvera,"[4i1b, 4i1a]"
4,Malkov,Razvoj sofvtvera,"[4i2a, 4r, 4i2b]"
5,Malkov,Razvoj softvera - vezbe,[4i1a]
6,Malkov,Razvoj softvera - vezbe,[4i1b]
7,Malkov,Razvoj softvera - vezbe,"[4i2a, 4r]"
8,Malkov,Razvoj softvera - vezbe,[4i2b]
9,Mitic,Istrazivanje podataka 2,[4i]


In [28]:
lecturers

,name,preffered_timeslots,not_preffered_timeslots,preffered_locations
0,Kartelj,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","{2, 1}"
1,Cukic,[],[],{}
2,Mitic,[],[],{}
3,Malkov,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]",{0}


In [29]:
calculate_fittnes(result_timetable, teaching_curriculum)

(0, 10)

In [445]:
start = datetime.now()
global_count = 0
timetable_v2 = init_timetable()
timetable_v2 = local_search_v2(timetable_v2, teaching_curriculum)
end = datetime.now()
 
# find difference loop start and end time and display
tdsoftext = (end - start).total_seconds()

count 0
drugi count 1
count 1
drugi count 1
count 1
drugi count 2
(49, 2)
(37, 2)
(28, 2)
(20, 4)
(15, 4)
(11, 4)
(7, 5)
(4, 5)
(3, 5)
(2, 5)
(1, 5)
(0, 6)
(0, 4)
(0, 3)
(0, 2)


In [422]:
start = datetime.now()
global_count = 0
timetable_v2 = init_timetable()
timetable_v2 = local_search_v2(timetable_v2, teaching_curriculum)
end = datetime.now()
 
# find difference loop start and end time and display
tdsoftext = (end - start).total_seconds()

(49, 2)
(37, 2)
(28, 2)
(20, 4)
(15, 4)
(11, 4)
(7, 5)
(4, 5)
(3, 5)
(2, 5)
(1, 5)
(0, 6)
(0, 4)
(0, 3)
(0, 2)


In [423]:
tdsoftext

48.075339

In [418]:
tdsoft

12.133904

In [410]:
tdrooms2

51.675677

In [397]:
tdgroups2

53.314199

In [386]:
tdgroups

72.694977

In [351]:
tdnewnew

85.288966

In [333]:
tdnew

91.944182

In [150]:
print('init', calculate_fittnes(timetable, teaching_curriculum))

IndexError: index 71 is out of bounds for axis 0 with size 12

In [168]:
start = datetime.now()
timetable_v2 = init_timetable()
timetable_v2 = local_search_v2(timetable_v2, teaching_curriculum)
end = datetime.now()
 
# find difference loop start and end time and display
td = (end - start).total_seconds()

(2520, 0)
(2481, 0)
(2443, 0)
(2407, 0)
(2372, 0)
(2338, 0)
(2306, 0)
(2275, 0)
(2244, 0)
(2214, 0)
(2184, 0)
(2154, 0)
(2126, 0)
(2098, 0)
(2072, 0)
(2046, 0)
(2020, 0)
(1995, 0)
(1971, 0)
(1947, 0)
(1924, 0)
(1901, 0)
(1878, 0)
(1856, 0)
(1834, 0)
(1812, 0)
(1790, 0)
(1769, 0)
(1748, 0)
(1727, 0)
(1706, 0)
(1686, 0)
(1666, 0)
(1647, 0)
(1628, 0)
(1609, 0)
(1590, 0)
(1571, 0)
(1552, 0)
(1534, 0)
(1516, 0)
(1498, 0)
(1480, 0)
(1462, 0)
(1444, 0)
(1427, 0)
(1410, 0)
(1393, 0)
(1376, 0)
(1359, 0)
(1343, 0)
(1327, 0)
(1311, 0)
(1295, 0)
(1279, 0)
(1264, 0)
(1249, 0)
(1234, 0)
(1219, 0)
(1204, 0)
(1190, 0)
(1176, 0)
(1162, 0)
(1148, 0)
(1134, 0)
(1120, 0)
(1106, 0)
(1092, 0)
(1079, 0)
(1066, 0)
(1053, 0)
(1040, 0)
(1027, 0)
(1015, 0)
(1003, 0)
(991, 0)
(979, 0)
(967, 0)
(955, 0)
(944, 0)
(933, 0)
(922, 0)
(911, 0)
(900, 0)
(890, 0)
(880, 0)
(870, 0)
(860, 0)
(850, 0)
(840, 0)
(830, 0)
(820, 0)
(810, 0)
(801, 0)
(792, 0)
(783, 0)
(774, 0)
(765, 0)
(756, 0)
(747, 0)
(738, 0)
(729, 0)
(721, 0

KeyboardInterrupt: 

In [170]:
best_timetable

NameError: name 'best_timetable' is not defined

In [110]:
td

6412.172493

In [99]:
td

59245.825527

In [58]:
create_helper_matrixes(timetable_v2, teaching_curriculum)

ValueError: invalid literal for int() with base 10: '{'

In [108]:
td

157.818532

In [223]:
td

97.26642

In [436]:
calculate_fittnes(timetable_v2, teaching_curriculum)

count 0
drugi count 1
count 1
drugi count 1
count 1
drugi count 2


(0, 2)

In [437]:
timetable_v2

,room,time_slot
0,0,12
1,1,3
2,0,0
3,3,14
4,4,3
5,5,16
6,1,0
7,7,26
8,8,29
9,9,48


In [389]:
start = datetime.now()
timetable_v3 = init_timetable()
timetable_v3 = local_search_v3(timetable_v3, teaching_curriculum)
end = datetime.now()
 
# find difference loop start and end time and display
td = (end - start).total_seconds()

(49, 7)
(37, 17)
(28, 22)
(21, 25)
(17, 26)
(14, 20)
(12, 20)
(11, 18)
(10, 18)
(9, 18)
(9, 15)
(9, 13)
(9, 12)
(9, 11)


In [68]:
calculate_fittnes(result_timetable, teaching_curriculum)

(0, 0)

In [70]:
result_timetable.to_csv('timetable.csv', index=True)

In [67]:
start = datetime.now()
k = 2
max_k = 10
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
while True:
    print('new iter')
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum, k)
    timetable = local_search_v2(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable = timetable
        k = 2
        print('found better score', current_fitness)
    else:
        k += 1
        print('increased k', k)
        if k >= max_k:
            print('algorithm finished')
            break

end = datetime.now()
 
# find difference loop start and end time and display
td = (end - start).total_seconds()

init (673, 9)
new iter
(632, 9)
(612, 10)
(593, 10)
(577, 8)
(561, 8)
(545, 8)
(530, 7)
(515, 6)
(501, 6)
(487, 6)
(473, 6)
(460, 6)
(447, 6)
(434, 6)
(422, 6)
(410, 6)
(399, 6)
(388, 6)
(378, 4)
(368, 4)
(358, 4)
(348, 4)
(339, 3)
(330, 3)
(321, 3)
(312, 3)
(303, 3)
(294, 3)
(285, 3)
(276, 3)
(268, 3)
(260, 3)
(252, 3)
(244, 3)
(236, 3)
(229, 2)
(222, 2)
(216, 2)
(210, 2)
(204, 2)
(198, 2)
(192, 2)
(186, 2)
(180, 2)
(174, 2)
(168, 2)
(162, 2)
(156, 2)
(151, 2)
(146, 2)
(141, 2)
(136, 2)
(131, 2)
(126, 2)
(121, 2)
(116, 2)
(111, 4)
(107, 2)
(103, 2)
(99, 2)
(95, 2)
(91, 2)
(87, 2)
(83, 2)
(79, 2)
(75, 2)
(71, 2)
(67, 2)
(63, 2)
(60, 1)
(57, 0)
(54, 0)
(51, 0)
(48, 0)
(45, 0)
(42, 0)
(39, 0)
(36, 0)
(33, 0)
(31, 0)
(29, 0)
(27, 0)
(25, 0)
(23, 0)
(21, 0)
(19, 0)
(17, 0)
(15, 0)
(13, 0)
(11, 0)
(9, 0)
(8, 0)
(7, 0)
(6, 0)
(5, 0)
(4, 0)
(3, 0)
(2, 0)
(1, 0)
(0, 0)
found better score (0, 0)


In [211]:
start = datetime.now()
k = 40
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
for i in range(10):
    print('iteration', i)
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search_v2(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable = timetable
end = datetime.now()
 
# find difference loop start and end time and display
td = (end - start).total_seconds()

init (2560, 53)
iteration 0
(2026, 51)
(1989, 51)
(1953, 51)
(1919, 51)
(1886, 51)
(1855, 49)
(1825, 47)
(1796, 46)
(1768, 45)
(1740, 45)
(1715, 45)
(1691, 45)
(1668, 45)
(1645, 46)
(1623, 46)
(1601, 46)
(1580, 46)
(1560, 46)
(1540, 46)
(1520, 46)
(1500, 46)
(1481, 46)
(1462, 46)
(1443, 46)
(1424, 46)
(1406, 45)
(1388, 44)
(1370, 44)
(1352, 44)
(1335, 42)
(1318, 40)
(1301, 39)
(1284, 39)
(1268, 37)
(1252, 36)
(1236, 36)
(1220, 36)
(1204, 36)
(1188, 36)
(1172, 37)
(1157, 36)
(1142, 36)
(1127, 36)
(1112, 36)
(1097, 37)
(1083, 35)
(1069, 35)
(1055, 35)
(1041, 35)
(1027, 35)
(1014, 33)
(1001, 32)
(988, 32)
(975, 32)
(962, 32)
(949, 32)
(936, 32)
(923, 34)
(911, 33)
(899, 33)
(887, 33)
(875, 33)
(864, 33)
(853, 33)
(842, 33)
(831, 33)
(820, 33)
(809, 33)
(799, 33)
(789, 33)
(779, 33)
(769, 33)
(759, 33)
(749, 33)
(740, 31)
(731, 31)
(722, 31)
(713, 31)
(704, 31)
(695, 31)
(686, 31)
(677, 31)
(669, 30)
(661, 30)
(653, 30)
(645, 30)
(637, 30)
(629, 30)
(621, 30)
(613, 31)
(606, 29)
(599, 27)


In [120]:
td

1946.039653

In [121]:
start = datetime.now()
k = 40
result_timetable2 = init_timetable()
print('init', calculate_fittnes(result_timetable2, teaching_curriculum))
for i in range(10):
    print('iteration', i)
    timetable = result_timetable2.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search_v3(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable2, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable2 = timetable
end = datetime.now()
 
# find difference loop start and end time and display
td2 = (end - start).total_seconds()

init (62, 2)
iteration 0
(4, 19)
(3, 19)
(2, 19)
(1, 21)
(0, 23)
(0, 21)
(0, 19)
(0, 18)
(0, 16)
(0, 15)
(0, 14)
iteration 1
(12, 29)
(10, 29)
(8, 30)
(7, 28)
(6, 27)
(5, 27)
(4, 27)
(3, 27)
(2, 27)
(1, 29)
(1, 27)
(1, 25)
(1, 24)
(1, 23)
(1, 22)
(1, 21)
iteration 2
(8, 34)
(6, 33)
(5, 32)
(4, 31)
(3, 31)
(2, 35)
(2, 29)
(2, 27)
(2, 26)
iteration 3
(4, 26)
(3, 24)
(2, 24)
(1, 24)
(0, 26)
(0, 24)
(0, 22)
(0, 20)
(0, 18)
(0, 17)
(0, 16)
(0, 15)
iteration 4
(3, 28)
(1, 37)
(0, 39)
(0, 37)
(0, 35)
(0, 33)
(0, 31)
(0, 29)
(0, 28)
(0, 27)
(0, 25)
(0, 20)
iteration 5
(7, 25)
(5, 27)
(4, 27)
(3, 27)
(2, 27)
(2, 25)
(2, 23)
(2, 21)
(2, 19)
(2, 17)
(2, 15)
(1, 18)
iteration 6
(8, 35)
(6, 44)
(5, 42)
(4, 40)
(3, 39)
(2, 38)
(1, 38)
(0, 38)
(0, 35)
(0, 32)
(0, 30)
(0, 28)
(0, 27)
(0, 26)
(0, 24)
iteration 7
(7, 34)
(6, 30)
(5, 28)
(4, 27)
(3, 27)
(2, 27)
(1, 29)
(0, 36)
(0, 33)
(0, 30)
(0, 28)
iteration 8
(5, 37)
(4, 35)
(3, 35)
(2, 35)
(1, 37)
(0, 41)
(0, 38)
(0, 36)
(0, 34)
(0, 32)
(0, 30)
(0, 2

In [122]:
td2

1236.490169

In [126]:
pip install py-heat-magic

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 42.2/42.2 MB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 6.5/6.5 MB 10.9 MB/s eta 0:00:00
     -------------------------------------- 154.7/154.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 121.0/121.0 kB 6.9 MB/s eta 0:00:00
     -------------------------------------- 137.8/137.8 kB 8.0 MB/s eta 0:00:00
     -------------------------------------- 163.6/163.6 kB 9.6 MB/s eta 0:00:00
     -------------------------------------- 965.4/965.4 kB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB ? eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 9.9 MB/s eta 0:00:00
     --

  DEPRECATION: pprofile is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: py-heat is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: py-heat-magic is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be f

In [127]:
%load_ext heat

In [130]:
%%heat
timetable_v2 = init_timetable()
timetable_v2 = local_search_v2(timetable_v2, teaching_curriculum)

NameError: name 'init_timetable' is not defined

In [131]:
pip install line_profiler

     ---------------------------------------- 84.7/84.7 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [111]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [112]:
timetable_v2 = init_timetable()

In [113]:
%lprun -f local_search_v2 local_search_v2(timetable_v2, teaching_curriculum)

(2520, 0)
(2481, 0)
(2443, 0)
(2407, -2)
(2372, -2)
(2338, -2)
(2306, -4)
(2275, -5)
(2244, -6)
(2214, -6)
(2184, -6)
(2154, -6)
(2126, -6)
(2098, -6)
(2072, -6)
(2046, -5)
(2020, -4)
(1995, -4)
(1971, -4)
(1947, -4)
(1924, -5)
(1901, -5)
(1878, -5)
(1856, -5)
(1834, -5)
(1812, -5)
(1790, -5)
(1769, -5)
(1748, -5)
(1727, -5)
(1706, -5)
(1686, -5)
(1666, -5)
(1647, -7)
(1628, -8)
(1609, -9)
(1590, -10)
(1571, -10)
(1552, -10)
(1534, -11)
(1516, -12)
(1498, -12)
(1480, -12)
(1462, -12)
(1444, -12)
(1427, -14)
(1410, -15)
(1393, -15)
(1376, -15)
(1359, -14)
(1343, -16)
(1327, -17)
(1311, -17)
(1295, -17)
(1279, -16)
(1264, -17)
(1249, -18)
(1234, -18)
(1219, -18)
(1204, -18)
(1190, -18)
(1176, -18)
(1162, -18)
(1148, -18)
(1134, -18)
(1120, -18)
(1106, -18)
(1092, -17)
(1079, -19)
(1066, -19)
(1053, -19)
(1040, -19)
(1027, -19)
(1015, -21)
(1003, -21)
(991, -21)
(979, -21)
(967, -21)
(955, -21)
(944, -22)
(933, -22)
(922, -22)
(911, -22)
(900, -22)
(890, -24)
(880, -24)
(870, -24)
(860, -

In [171]:
def test():
    print(teaching_curriculum.iloc[5]['groups'])
    print(teaching_curriculum.iat[5,2])

In [172]:
test()

{'1'}
{'1'}


In [173]:
%lprun -f test test()

{'1'}
{'1'}


In [22]:
def create_helper_matrixes(timetable, teaching_curriculum):
    lecturer_matrix_helper = np.empty((lecturers.shape[0], len(time_slots)), dtype=object)
    for i in range(lecturer_matrix_helper.shape[0]):
        for j in range(lecturer_matrix_helper.shape[1]):
            lecturer_matrix_helper[i,j] = set()
    groups_matrix_helper = np.empty((groups.shape[0], len(time_slots)), dtype=object)
    for i in range(groups_matrix_helper.shape[0]):
        for j in range(groups_matrix_helper.shape[1]):
            groups_matrix_helper[i,j] = set()
    rooms_matrix_helper = np.empty((rooms.shape[0], len(time_slots)), dtype=object)
    for i in range(rooms_matrix_helper.shape[0]):
        for j in range(rooms_matrix_helper.shape[1]):
            rooms_matrix_helper[i,j] = set()
    for i in range(timetable.shape[0]):
        timeslot = timetable.iat[i,1]
        num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
        for j in range(timeslot, timeslot + num_of_classes):
            lecturer_matrix_helper[int(teaching_curriculum.iat[i,0]),j].add(str(i))
            for group in teaching_curriculum.iat[i,2]:
                groups_matrix_helper[int(group),j].add(str(i))
            rooms_matrix_helper[int(timetable.iat[i,0]),j].add(str(i))
    return (lecturer_matrix_helper, groups_matrix_helper, rooms_matrix_helper)

In [65]:
teaching_curriculum['groups'] = teaching_curriculum['groups'].apply(eval)

In [66]:
create_helper_matrixes(timetable_v2, teaching_curriculum)

{0}
{0}
{1}
{1}
{1}
{2}
{2}
{2}
{4}
{4}
{4}
{5}
{5}
{5}
{7}
{7}
{7}
{8}
{8}
{8}
{10}
{10}
{10}
{11}
{11}
{11}
{0}
{0}
{1}
{1}
{1}
{2}
{2}
{2}
{4}
{4}
{4}
{5}
{5}
{5}
{7}
{7}
{7}
{8}
{8}
{8}
{10}
{10}
{10}
{11}
{11}
{11}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{28, 22}
{28, 22}
{28, 22}
{28, 22}
{29, 23}
{29, 23}
{29, 23}
{29, 23}
{25, 31}
{25, 31}
{25, 31}
{25, 31}
{32, 26}
{32, 26}
{32, 26}
{32, 26}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{42, 85, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{18, 12, 15}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{16, 19, 13}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{17, 20, 14}
{28, 22}
{28

(array([[set(), set(), {'62'}, ..., set(), set(), set()],
        [set(), set(), set(), ..., set(), set(), set()],
        [set(), set(), {'54', '55'}, ..., set(), set(), set()],
        ...,
        [set(), {'44'}, {'44'}, ..., set(), set(), set()],
        [set(), {'53'}, {'53'}, ..., set(), set(), set()],
        [set(), set(), set(), ..., set(), set(), set()]], dtype=object),
 array([[{'9', '0'}, {'9', '0'}, set(), ..., set(), set(), set()],
        [{'1', '10'}, {'1', '10'}, {'1', '10'}, ..., set(), set(), set()],
        [{'2', '11'}, {'2', '11'}, {'2', '11'}, ..., set(), set(), set()],
        ...,
        [set(), set(), set(), ..., set(), set(), set()],
        [set(), set(), set(), ..., set(), set(), set()],
        [set(), set(), set(), ..., set(), set(), set()]], dtype=object),
 array([[{'0'}, {'0', '27'}, {'54', '27'}, ..., set(), set(), set()],
        [{'1'}, {'1', '28'}, {'1', '28', '55'}, ..., set(), set(), set()],
        [{'2'}, {'2', '29'}, {'2', '56', '29'}, ..., se

In [67]:
teaching_curriculum.dtypes

lecturer            int64
class               int64
groups             object
groups_to_check    object
dtype: object

In [23]:
def check_lecturer_constraint_delta(timetable, teaching_curriculum, lecturer_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for j in range(old_timeslot, old_timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if old_timeslot % 13 > 0:
        for el in lecturer_matrix_helper[lecturer_index, old_timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    if (old_timeslot + num_of_classes - 1) % 13 < 12:
        for el in lecturer_matrix_helper[lecturer_index, old_timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

def check_lecturer_constraint_delta_curr(timetable, teaching_curriculum, lecturer_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for j in range(timeslot, timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for el in lecturer_matrix_helper[lecturer_index, timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for el in lecturer_matrix_helper[lecturer_index, timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

In [24]:
def check_lecturer_constraint_delta_room(timetable, teaching_curriculum, lecturer_matrix_helper, i, room):
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    new_room_location = rooms.iat[int(room),1]
    if new_room_location == room_location:
        return 0
    previous_constraint_count = 0
    timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for j in range(timeslot, timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for el in lecturer_matrix_helper[lecturer_index, timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for el in lecturer_matrix_helper[lecturer_index, timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

def check_lecturer_constraint_delta_curr_room(timetable, teaching_curriculum, lecturer_matrix_helper, i, room):
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    new_room_location = rooms.iat[int(room),1]
    if new_room_location == room_location:
        return 0
    previous_constraint_count = 0
    timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for j in range(timeslot, timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for el in lecturer_matrix_helper[lecturer_index, timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != new_room_location:
                set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for el in lecturer_matrix_helper[lecturer_index, timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != new_room_location:
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

In [25]:
def check_groups_constraint_delta(timetable, teaching_curriculum, groups_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    groups_indexes = teaching_curriculum.iat[i,3]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for group_index in groups_indexes:
        for j in range(old_timeslot, old_timeslot + num_of_classes):
            for el in groups_matrix_helper[int(group_index), j]:
                if el != str(i):
                    set_of_intersections.add(el)
    if old_timeslot % 13 > 0:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), old_timeslot-1]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                    set_of_location_intersections.add(el)
    if (old_timeslot + num_of_classes - 1) % 13 < 12:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), old_timeslot + num_of_classes]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                    set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

def check_groups_constraint_delta_curr(timetable, teaching_curriculum, groups_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    groups_indexes = teaching_curriculum.iat[i,3]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for group_index in groups_indexes:
        for j in range(timeslot, timeslot + num_of_classes):
            for el in groups_matrix_helper[int(group_index), j]:
                if el != str(i):
                    set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot-1]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                    set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot + num_of_classes]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                    set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

In [26]:
def check_groups_constraint_delta_room(timetable, teaching_curriculum, groups_matrix_helper, i, room):
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    new_room_location = rooms.iat[int(room),1]
    if new_room_location == room_location:
        return 0
    previous_constraint_count = 0
    timeslot = timetable.iat[i,1]
    groups_indexes = teaching_curriculum.iat[i,3]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for group_index in groups_indexes:
        for j in range(timeslot, timeslot + num_of_classes):
            for el in groups_matrix_helper[int(group_index), j]:
                if el != str(i):
                    set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot-1]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                    set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot + num_of_classes]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                    set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

def check_groups_constraint_delta_curr_room(timetable, teaching_curriculum, groups_matrix_helper, i, room):
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    new_room_location = rooms.iat[int(room),1]
    if new_room_location == room_location:
        return 0
    previous_constraint_count = 0
    timeslot = timetable.iat[i,1]
    groups_indexes = teaching_curriculum.iat[i,3]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for group_index in groups_indexes:
        for j in range(timeslot, timeslot + num_of_classes):
            for el in groups_matrix_helper[int(group_index), j]:
                if el != str(i):
                    set_of_intersections.add(el)
    if timeslot % 13 > 0:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot-1]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != new_room_location:
                    set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 13 < 12:
        for group_index in groups_indexes:
            for el in groups_matrix_helper[int(group_index), timeslot + num_of_classes]:
                if rooms.iat[int(timetable.iat[int(el),0]),1] != new_room_location:
                    set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

In [27]:
def check_room_constraint_delta(timetable, teaching_curriculum, room_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    room_index = int(timetable.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    for j in range(old_timeslot, old_timeslot + num_of_classes):
        for el in room_matrix_helper[room_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    previous_constraint_count = len(set_of_intersections)
    return previous_constraint_count

def check_room_constraint_delta_curr(timetable, teaching_curriculum, room_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    room_index = int(timetable.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for j in range(timeslot, timeslot + num_of_classes):
        for el in room_matrix_helper[room_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    previous_constraint_count = len(set_of_intersections)
    return previous_constraint_count

In [28]:
def check_room_constraint_delta_room(timetable, teaching_curriculum, room_matrix_helper, i, room):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    room_index = int(timetable.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    for j in range(old_timeslot, old_timeslot + num_of_classes):
        for el in room_matrix_helper[room_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    previous_constraint_count = len(set_of_intersections)
    return previous_constraint_count

def check_room_constraint_delta_curr_room(timetable, teaching_curriculum, room_matrix_helper, i, room):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    for j in range(old_timeslot, old_timeslot + num_of_classes):
        for el in room_matrix_helper[int(room), j]:
            if el != str(i):
                set_of_intersections.add(el)
    previous_constraint_count = len(set_of_intersections)
    return previous_constraint_count

In [29]:
def check_lecturer_constraint_delta(timetable, teaching_curriculum, lecturer_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for j in range(old_timeslot, old_timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if old_timeslot % 12 > 0:
        for el in lecturer_matrix_helper[lecturer_index, old_timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    if (old_timeslot + num_of_classes - 1) % 12 < 11:
        for el in lecturer_matrix_helper[lecturer_index, old_timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location:
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

def check_lecturer_constraint_delta_curr(timetable, teaching_curriculum, lecturer_matrix_helper, i, timeslot):
    previous_constraint_count = 0
    old_timeslot = timetable.iat[i,1]
    lecturer_index = int(teaching_curriculum.iat[i,0])
    num_of_classes = classes.iat[int(teaching_curriculum.iat[i,1]),2]
    set_of_intersections = set()
    set_of_location_intersections = set()
    room_location = rooms.iat[int(timetable.iat[i,0]),1]
    for j in range(timeslot, timeslot + num_of_classes):
        for el in lecturer_matrix_helper[lecturer_index, j]:
            if el != str(i):
                set_of_intersections.add(el)
    if timeslot % 12 > 0:
        for el in lecturer_matrix_helper[lecturer_index, timeslot-1]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                set_of_location_intersections.add(el)
    if (timeslot + num_of_classes - 1) % 12 < 11:
        for el in lecturer_matrix_helper[lecturer_index, timeslot + num_of_classes]:
            if rooms.iat[int(timetable.iat[int(el),0]),1] != room_location and el != str(i):
                set_of_location_intersections.add(el)
    previous_constraint_count = len(set_of_intersections) + len(set_of_location_intersections.difference(set_of_intersections))
    return previous_constraint_count

In [379]:
helper_matrixes = create_helper_matrixes(timetable, teaching_curriculum)

In [383]:
check_groups_constraint_delta_curr(timetable, teaching_curriculum, helper_matrixes[1], 1, 36)

0

In [364]:
global_count 

0

In [327]:
start = datetime.now()
check_lecturer_constraint_delta(timetable, teaching_curriculum, helper_matrixes[0], 8, 26)
end = datetime.now()
 
# find difference loop start and end time and display
td1 = (end - start).total_seconds()

In [328]:
start = datetime.now()
lecturer_constraint_delta(timetable, teaching_curriculum, 8)
end = datetime.now()
 
# find difference loop start and end time and display
td2 = (end - start).total_seconds()

In [329]:
td1

0.001133

In [330]:
td2

0.002382

In [312]:
timetable

,room,time_slot
0,0,39
1,0,36
2,10,30
3,0,41
4,5,6
5,0,19
6,0,3
7,1,16
8,0,16
9,13,28


In [269]:
mat = np.empty((lecturers.shape[0], len(time_slots)), dtype=object)
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        mat[i,j] = set()
mat[0,0].add('a')
mat

array([[{'a'}, set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set()],
       [set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set(), set(), set(), set(),
        set(), set(), set(), set(), set(), set()],
       [set(), set(), set(), set(), set(), set

In [262]:
np.full((5,5), {'a'})

array([[{'a'}, {'a'}, {'a'}, {'a'}, {'a'}],
       [{'a'}, {'a'}, {'a'}, {'a'}, {'a'}],
       [{'a'}, {'a'}, {'a'}, {'a'}, {'a'}],
       [{'a'}, {'a'}, {'a'}, {'a'}, {'a'}],
       [{'a'}, {'a'}, {'a'}, {'a'}, {'a'}]], dtype=object)

In [61]:
start = datetime.now()
k = 40
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))
for i in range(10):
    print('iteration', i)
    timetable = result_timetable.copy()
    timetable = shaking(timetable, teaching_curriculum)
    timetable = local_search_v2(timetable, teaching_curriculum)
    current_fitness = calculate_fittnes(timetable, teaching_curriculum)
    current_best_fitness = calculate_fittnes(result_timetable, teaching_curriculum)
    if current_fitness[0] < current_best_fitness[0] or (current_fitness[0] == current_best_fitness[0] and current_fitness[1] < current_best_fitness[1]):
        result_timetable = timetable
end = datetime.now()
 
# find difference loop start and end time and display
td = (end - start).total_seconds()

init (352, 4)
iteration 0
(121, 9)
(112, 9)
(104, 9)
(96, 9)
(88, 9)
(80, 9)
(74, 8)
(68, 8)
(63, 8)
(58, 8)
(53, 8)
(48, 8)
(43, 8)
(39, 8)
(35, 8)
(31, 9)
(28, 9)
(25, 9)
(22, 9)
(19, 9)
(16, 10)
(14, 8)
(12, 8)
(11, 5)
(10, 3)
(9, 2)
(8, 2)
(7, 2)
(6, 2)
(5, 2)
(4, 2)
(3, 2)
(2, 2)
(1, 2)
(0, 2)
(0, 0)
iteration 1
(22, 12)
(19, 12)
(16, 12)
(14, 10)
(12, 9)
(10, 9)
(8, 9)
(7, 7)
(6, 6)
(5, 5)
(4, 4)
(3, 4)
(2, 4)
(1, 4)
(0, 4)
(0, 2)
(0, 1)
(0, 0)
iteration 2
(22, 9)
(19, 9)
(16, 9)
(14, 7)
(12, 7)
(10, 7)
(8, 7)
(6, 7)
(4, 8)
(3, 7)
(2, 7)
(1, 7)
(0, 7)
(0, 4)
(0, 2)
(0, 1)
(0, 0)
iteration 3
(20, 8)
(17, 8)
(14, 8)
(11, 8)
(9, 8)
(7, 8)
(5, 8)
(4, 6)
(3, 6)
(2, 6)
(1, 6)
(0, 6)
(0, 4)
(0, 2)
(0, 0)
iteration 4
(17, 12)
(14, 12)
(12, 10)
(10, 9)
(8, 9)
(6, 9)
(5, 7)
(4, 5)
(3, 5)
(2, 5)
(1, 5)
(0, 5)
(0, 3)
(0, 1)
(0, 0)
iteration 5
(28, 7)
(25, 6)
(22, 6)
(20, 4)
(18, 3)
(16, 2)
(14, 2)
(12, 2)
(10, 2)
(8, 4)
(7, 4)
(6, 4)
(5, 4)
(4, 4)
(3, 4)
(2, 4)
(1, 4)
(0, 4)
(0, 2)
(0, 0)
it

In [62]:
td

1131.971241

In [59]:
result_timetable = init_timetable()
print('init', calculate_fittnes(result_timetable, teaching_curriculum))

a
b
a
b
a
count 0
drugi count 0
a
count 0
drugi count 0
a
count 0
drugi count 0
a
b
a
b
a
b
a
count 0
drugi count 0
a
count 0
drugi count 0
a
b
a
count 0
drugi count 0
a
b
a
b
a
count 0
drugi count 0
a
b
a
b
a
b
a
b
a
b
a
count 0
drugi count 0
a
count 0
drugi count 0
a
count 2
drugi count 2
a
b
a
b
a
b
a
b
a
count 3
drugi count 3
a
count 4
drugi count 4
a
b
a
count 4
drugi count 4
a
b
a
count 4
drugi count 4
a
count 4
drugi count 4
a
count 4
drugi count 4
a
b
a
b
a
b
a
b
a
b
a
b
a
count 4
drugi count 4
a
b
a
count 4
drugi count 4
a
count 4
drugi count 4
a
b
a
b
a
count 4
drugi count 4
a
b
a
b
a
count 4
drugi count 4
a
b
a
b
a
b
a
count 4
drugi count 4
init (352, 4)


In [33]:
teaching_curriculum

,lecturer,class,groups,groups_to_check
0,41,19,{75},{75}
1,34,10,{75},{75}
2,21,67,{54},{54}
3,22,33,{30},{30}
4,27,44,{54},{54}
...,...,...,...,...
68,2,59,{51},{51}
69,34,119,{51},{51}
70,3,127,{51},{51}
71,27,73,{51},{51}


In [41]:
result_timetable

,room,time_slot
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
68,12,4
69,13,4
70,0,5
71,1,5


In [68]:
result_timetable.iloc[60:]

,room,time_slot
60,4,18
61,5,6
62,6,4
63,7,12
64,8,16
65,9,14
66,11,50
67,5,0
68,3,3
69,13,26


In [67]:
teaching_curriculum_copy.iloc[60:]

,lecturer,class,groups,groups_to_check
60,Милан Банковић,Р306,[5ир],{53}
61,Младен Николић,Р308,[5ир],{53}
62,Мирко Спасић,МФАСП,[П1],{51}
63,Ненад Митић,МФИП,[П1],{51}
64,Младен Николић,МФМУ,[П1],{51}
65,Александар Картељ,МФРИ,[П1],{51}
66,Јована Ковачевић,МФУБИ,[П1],{51}
67,Јована Ковачевић,ХФУПБ,[П1],{51}
68,Александар Картељ,УУХЈВИ,[П1],{51}
69,Сана Стојановић Ђурђевић,УУААВИ,[П1],{51}


In [79]:
print('init', calculate_fittnes(result_timetable, teaching_curriculum))

IndexError: index 0 is out of bounds for axis 0 with size 0